In [1]:
%load_ext autoreload
%autoreload 2

from common import add_workspace_to_path

add_workspace_to_path()

In [2]:
from cloud.dependencies.clients import resolve_gmail_client


gm = resolve_gmail_client()

In [3]:
from sebastian.clients.google.gmail.query_builder import GmailQueryBuilder
from datetime import datetime


query = GmailQueryBuilder().subject("Ihre winSIM-Rechnung", exact=False).after_date(datetime(2025, 10, 1)).build()
query

'subject:Ihre winSIM-Rechnung after:1759276800'

In [4]:
messages = gm.fetch_mails(query=query)

In [5]:
len(messages)

2

In [6]:
message = messages[0]

In [7]:
assert "parts" in message.raw_payload

In [8]:
parts = message.raw_payload["parts"]
parts

[{'partId': '0',
  'mimeType': 'text/html',
  'filename': '',
  'headers': [{'name': 'Content-Type',
    'value': 'text/html; charset="ISO-8859-1"'},
   {'name': 'Content-Transfer-Encoding', 'value': 'quoted-printable'}],
  'body': {'size': 7894,
   'data': 'PCFET0NUWVBFIGh0bWw-CjxodG1sIGxhbmc9ImVuIiB4bWxuczpvPSJ1cm46c2NoZW1hcy1taWNyb3NvZnQtY29tOm9mZmljZTpvZmZpY2UiIHhtbG5zOnY9InVybjpzY2hlbWFzLW1pY3Jvc29mdC1jb206dm1sIj4KPGhlYWQ-PC9oZWFkPgo8aGVhZD4KICA8bWV0YSBjaGFyc2V0PSJVVEYtOCI-CiAgPG1ldGEgaHR0cC1lcXVpdj0iQ29udGVudC1UeXBlIiBjb250ZW50PSJ0ZXh0L2h0bWw7IGNoYXJzZXQ9dXRmLTgiPgogIDwhLS1baWYgIW1zb10-PCEtLT4KICA8bWV0YSBodHRwLWVxdWl2PSJYLVVBLUNvbXBhdGlibGUiIGNvbnRlbnQ9IklFPWVkZ2UiPgogIDwhLS08IVtlbmRpZl0tLT4KICA8bWV0YSBuYW1lPSJ2aWV3cG9ydCIgY29udGVudD0id2lkdGg9ZGV2aWNlLXdpZHRoLCBpbml0aWFsLXNjYWxlPTEuMCI-CiAgPG1ldGEgbmFtZT0iZm9ybWF0LWRldGVjdGlvbiIgY29udGVudD0idGVsZXBob25lPW5vLCBkYXRlPW5vLCBhZGRyZXNzPW5vLCBlbWFpbD1ubyIgLz4KICA8dGl0bGU-d2luU0lNIFJlY2hudW5nPC90aXRsZT4KPCEtLVtpZiBtc29dPgo8eG1sPgo8bzpPZ

In [9]:
from pydantic import BaseModel, ConfigDict, Field

class PdfMessageBody(BaseModel):
    model_config = ConfigDict(extra="ignore")
    
    attachment_id: str = Field(alias="attachmentId")
    size: int

class PdfMessagePart(BaseModel):
    model_config = ConfigDict(extra="ignore")
    
    filename: str
    mime_type: str = Field(alias="mimeType")
    body: PdfMessageBody

In [10]:
pdf_parts = [part for part in parts if part.get("filename", "").endswith(".pdf")]
pdf_parts

[{'partId': '1',
  'mimeType': 'application/pdf',
  'filename': 'INVB868113177.pdf',
  'headers': [{'name': 'Content-Type',
    'value': 'application/pdf; name="INVB868113177.pdf"'},
   {'name': 'Content-Transfer-Encoding', 'value': 'base64'},
   {'name': 'Content-Disposition',
    'value': 'attachment; filename="INVB868113177.pdf"'}],
  'body': {'attachmentId': 'ANGjdJ8zHMGfyEGfIobwuviu_JM8njCcHL4ON9JKCwduhlQLfENPCOGq-p2Ba1e6fvmLk70gtlGelb24pBA6-74Y7nZT2mH3MvDUusVTQcCNWx9iokK0K7jEJ04aV8jZ46cZuuM3kM4M49A7D3ids_idQU9F2TMrUGJ-kPRB6b4M02Vd7Wm1E1hxJvIuJez_wgVS8h79TpZN5jHTJcgPfQN4zH8S3Yw1H7Ds7WvbQm_ov1H5IFaO3YvtOwQm-ii1-p4hUfcDhgGyrjphHfraJhWUkTYk-1OBeE8m3La_eZEVpoff0-36ElQXks_S7vAcCqPhBU7LamdqpX3ZkbLLvwzkwVRU3sKO8XnuNlswu5iN9p6OKK7wovrL0CdnG4V8lawoJZXUT5vZRvsfBshVp2h__df2NmhtXiqr4GdpVg',
   'size': 10907}}]

In [11]:
PdfMessagePart.model_validate(pdf_parts[0])

PdfMessagePart(filename='INVB868113177.pdf', mime_type='application/pdf', body=PdfMessageBody(attachment_id='ANGjdJ8zHMGfyEGfIobwuviu_JM8njCcHL4ON9JKCwduhlQLfENPCOGq-p2Ba1e6fvmLk70gtlGelb24pBA6-74Y7nZT2mH3MvDUusVTQcCNWx9iokK0K7jEJ04aV8jZ46cZuuM3kM4M49A7D3ids_idQU9F2TMrUGJ-kPRB6b4M02Vd7Wm1E1hxJvIuJez_wgVS8h79TpZN5jHTJcgPfQN4zH8S3Yw1H7Ds7WvbQm_ov1H5IFaO3YvtOwQm-ii1-p4hUfcDhgGyrjphHfraJhWUkTYk-1OBeE8m3La_eZEVpoff0-36ElQXks_S7vAcCqPhBU7LamdqpX3ZkbLLvwzkwVRU3sKO8XnuNlswu5iN9p6OKK7wovrL0CdnG4V8lawoJZXUT5vZRvsfBshVp2h__df2NmhtXiqr4GdpVg', size=10907))

In [12]:
import base64


def download_pdf_attachment(service, message_id:str, attachment_id:str) -> bytes:
    attachment = (
        service.users()  # type: ignore
        .messages()
        .attachments()
        .get(
            userId="me",
            messageId=message_id,
            id=attachment_id,
        )
        .execute()
    )

    if not attachment:
        raise ValueError("Attachment not found")

    return base64.urlsafe_b64decode(attachment["data"])
        

In [13]:
testi = download_pdf_attachment(
    gm._service,
    message.id,
    pdf_parts[0]["body"]["attachmentId"])

This is what the method should return:

In [14]:
from io import BytesIO


b = BytesIO(testi)


This is only for manual validation in the notebook:

In [15]:

with open("output.pdf", "wb") as f:
    f.write(b.getvalue())

## Using the new download_pdf_attachments method

In [16]:
pdf_attachments = gm.download_pdf_attachments(query=query)
pdf_attachments

[PdfAttachment(filename='INVB868113177.pdf', data=<_io.BytesIO object at 0x7fce554eac50>),
 PdfAttachment(filename='INVB859802187.pdf', data=<_io.BytesIO object at 0x7fce554e9760>)]

In [18]:
# Verify it's a PdfAttachment object with filename and data
assert isinstance(pdf_attachments[0].data, BytesIO)
print(f"Filename: {pdf_attachments[0].filename}")
print(f"Data type: {type(pdf_attachments[0].data)}")

Filename: INVB868113177.pdf
Data type: <class '_io.BytesIO'>
